In [ ]:
# @title preprocessing

import os
import torch
# !pip install clip
import clip
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.images = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.images.sort()  # Ensure the order is consistent

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, img_path

def create_dataloader(directory, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((1000, 1000)),  # Assuming all images are resized to 1000x1000
        preprocess
    ])
    dataset = CustomDataset(directory, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return dataloader

EMOTIONS = list(sorted(['anger', 'disgust', 'fear', 'happiness', 'sadness']))


In [ ]:
# @title evaluation
# Create DataLoaders for input and output images
input_image_dir = '/content/with-beautiful-colors_resized_frame_1.jpeg'
input_dataloader = create_dataloader(input_image_dir, batch_size=1)

output_image_dir = '/content/content/output_folder/with beautiful colors_resized_frame_1_anger-1.jpg'
output_dataloader = create_dataloader(output_image_dir, batch_size=1)

for (input_image, input_path), (output_image, output_path) in zip(input_dataloader, output_dataloader):
        input_image = input_image.to(device)
        output_image = output_image.to(device)
        # Compute embeddings
        with torch.no_grad():
            input_embedding = model.encode_image(input_image)
            output_embedding = model.encode_image(output_image)
for emotion in EMOTIONS:
    # Process each pair of input and output images




        # Compute cosine similarity
        cosine_sim = torch.nn.functional.cosine_similarity(input_embedding, output_embedding, dim=1)

        # Print or save the similarity score
        print(f"Comparing {input_path[0]} with {output_path[0]}: Cosine Similarity = {cosine_sim.item()}")

In [ ]:
# import zipfile
# !unzip '/content/output_folder.zip'
